# Kinect Monte Carlo Simulations Using LAMMPS

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.rst)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the `bin` subfolder for the path that you have provided for the installation step above

## Setting the environment

We will use [matplotlib](https://matplotlib.org/), [seaborn](https://seaborn.pydata.org/) and [pandas](https://pandas.pydata.org/) libraries for plotting. You can install it using [pip](https://pip.pypa.io/en/stable/) like

In [ ]:
!pip install seaborn --user

### Notes
> * The `${VOTCASHARE}` environmental variable is set to the path that you provided during the VOTCA installation, by the default is set to `/usr/local/votca`.
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

##  Check the mapping

Let us first output `.pdb` files for the segments, qmmolecules and classical segments in order to check the mapping.

We need to update the `mapchecker` options.

Now we can run the checker as follows,

In [ ]:
!xtp_run -e mapchecker -c map_file=system.xml -f state.hdf5

## Site energies and pair energy differences
We will compute the histrogram using `resolution_sites` of 0.03 eV. See [eanalyze options and defaults](https://votca.github.io/xtp/eanalyze.html) for more information.

In [ ]:
!xtp_run -e eanalyze -c resolution_sites=0.03 -f state.hdf5

In the current work directoy you can see the resulting files,

In [ ]:
!ls eanalyze*

## Plotting the energies
We will the previously installed `pandas` and `seaborn` library to plot the electron histrogram computed in the previous step,

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
columns = ["E(eV)", "counts"] 
df = pd.read_table("eanalyze.pairhist_e.out", comment="#", sep='\s+',names=columns, skiprows=2) 
sns.relplot(x="E(eV)", y="counts", ci=None, kind="line", data=df) 
plt.plot()

## Couplings histrogram
In this step we will analyze the electron/hole couplings, using the `ianalyze` calculator using the `resolution_logJ2` parameter of 0.1 units. See the [ianalyze options and defaults](https://votca.github.io/xtp/ianalyze.html) for more information about the calculator.

In [ ]:
!xtp_run -e ianalyze -c resolution_logJ2=0.1 states=e,h -f state.hdf5

## Plotting the coupling histogram
We can now plot the logarithm of the squared coupling for the hole,

In [ ]:
columns = ["logJ2", "counts"] 
df = pd.read_table("ianalyze.ihist_h.out", comment="#", sep='\s+',names=columns, skiprows=2) 
sns.relplot(x="logJ2", y="counts", ci=None, kind="line", data=df) 
plt.plot()

## KMC simulations of multiple holes or electrons in periodic boundary conditions
Finally, lets do a 1000 seconds KMC simulation for the electron, with a 10 seconds window between output and a field of 10 *V/m* along the x-axis,

In [ ]:
!xtp_run -e kmcmultiple -c runtime=1000 outputtime=10 field=10,0,0 carriertype=electron -f state.hdf5

You can find both the *occupation data* and the *rates* for the electron at 300 K, on files `occupation.dat`  and `rates.dat`, respectively.